# Hands-on Lab 8: Controls Part 2

### EECS 16B: Designing Information Devices and Systems II, Fall 2022

Written by Vivek Athalye and Andrew Blatner (2017). Revised by Peter Schafhalter (2019).

Updated by Megan Zeng, Steven Lu (2022)

Updated by Mingyang Wang, Shrey Aeron, and Megan Zeng (2022)

## Table of Contents

* [Controls Part 1 Lab Note](https://eecs16b.org/lab/notes/lab_note7.pdf)
* [Introduction](#intro)
* [Part 1: Analyzing a Turn](#part1)
* [Part 2: Implementing Turns](#part2)
* [Part 3: Correcting Mechanical Errors](#part3)
* [Part 4: Micboard Verification](#part4)

<a id='intro'></a>
## Introduction

In the previous lab, we developed a closed-loop controller to make the car go straight. However, we also want the car to be able to turn! In this lab, we will modify the control scheme from Lab 7 to enable the car to turn. For a refresher on our closed-loop controller, please refer to the Controls Part 1 lab note [here](https://eecs16b.org/lab/notes/lab_note7.pdf). There is no lab note for Lab 8: everything you need is in this notebook!

## Optional Pre-Lab

Complete the [(optional) pre-lab assignment](https://www.gradescope.com/courses/414337/assignments/2396880) before doing the lab.

<a id='part1'></a>
## <span style="color:navy">Part 1: Analyzing a Turn</span>

### Materials
- S1XT33N Car
- Closed-loop control scheme

First, we will try to build intuition for how we implement turning via closed-loop control. Suppose we want to turn with a specific radius.

### **We have been turning during Closed-Loop Control!**

We can get a clue about turning by considering how we implemented closed-loop control. We perform feedback control to minimize the difference between the two wheels ($\delta[i] = d_L[i] - d_R[i]$). When perturbations cause one wheel to get ahead of the other, the result is a non-zero $\delta$, causing feedback control to turn the car to correct the error.

Our feedback control policy is: 
$$u_L[i] = u_L^{OL} - \frac{f_L}{\theta_L}\delta[i]$$

$$u_R[i] = u_R^{OL} + \frac{f_R}{\theta_R}\delta[i]$$


Let's say the right wheel has moved further than the left ($d_R[i] > d_L[i]$), resulting in a negative $\delta[i]$. A negative $\delta[i]$ results in a reduction of $u_R[i]$, and an increase of $u_L[i]$. But what does this mean physically?

If the right wheel is getting ahead of the left one, the left wheel will need to move a little faster and the right wheel a little slower for the wheels to even back out. Thus, feedback control corrects errors by turning to compensate for them.  

So we know that $\delta \ne 0$ can be used to turn the car. What if we artificially modified our $\delta[i]$ values to make the car think it was turning?

### **Turning via reference tracking**

To turn, we will manually add some $\delta_{ref}$ to our $\delta$ value at each step of the control scheme. By doing so, we are tricking our control scheme into thinking that our car has turned by some amount to generate that extra $\delta_{ref}$ of error, causing it to turn in the opposite direction to compensate. When making a turn, the magnitude of $\delta$ grows because otherwise, the car is driving straighter. Therefore, we need to update $\delta_{ref}$ at each timestep to ensure that the car continues to turn.

Without loss of generality, we'll analyze a right turn, corresponding to adding a **negative** $\delta_{ref}$ value (the car thinks it's turned left, so it tries to turn right). Our goal is to generate this $\delta_{ref}$. 

We would like the car to turn with a specified radius $r$ and linear velocity $v^*$. The controller's unit for distance is encoder ticks, but each tick is approximately 1 cm of wheel circumference. Additionally, we want our car to turn gradually (rather than making a pivot turn), so $\delta_{ref}$ will be a function of the controller's time-step.

We define the following variables:
- $i$ [time] - timestep
- $r$ [cm] - turn radius of the center of the car; 1 cm $\approx$ 1 encoder tick
- $d$ [tick] - distance traveled by the center of the car
- $l$ [cm] - distance between the centers of the wheels; 1 cm $\approx$ 1 encoder tick
- $\omega$ [rad/time] - angular velocity
- $\theta$ [rad] - angle turned

Inspect the following diagram:

<center>
<img width="400px" src="images/turning_figure.png">
</center>


From this geometry, can write $\delta_{ref}[i]$ in the following form?
$$\delta_{ref}[i] = f(r,v^*,l,i)$$

Note that Arc length = radius * central angle.

Hint 1:  Write $d_l$ and $d_r$ in terms of their radii and $\theta$.
<br>
Hint 2: How can we relate $v*$ to $\theta$?


<h3><span style='color:red'>**Check your answer with lab staff!**</span></h3>

**Answer:**

<a id='part2'></a>
## <span style="color:navy">Part 2: Implementing Turns</span>

### Materials
- S1XT33N Car
- Closed-loop control scheme


### Sampling periods 
 
 Our implementations of data collection and control loop have different time periods: data collection collected data on half-second intervals, while our controller updates the trajectory every 0.1 seconds. This means that our controller will be running at a faster rate than our data collection. However, our definition of the time step is dependent on the sampling period we use, so we will need to account for this difference by adjusting $v^*$ in our implementation of turning.

Let the sampling periods of the control loop and the data collection be $T_c$ and $T_d$, which have the units $\frac{s}{sample}$. Therefore,their respective sampling frequencies $F_c = \frac{1}{T_c}$ and $F_d = \frac{1}{T_d}$ have the units $\frac{samples}{s}$. We define $m=\frac{F_c}{F_d}$, the ratio of the sampling frequencies.

Our controller will interpret velocity in terms of distance traveled (measured in encoder ticks) per sample ($\frac{ticks}{sample}$). However, we need to ensure the car is travelling at the same ground velocity, which is measured per second ($\frac{ticks}{s}$). To get a constant $\frac{ticks}{s}$, we need $v^*_c \cdot F_c = v^* \cdot F_d$.

Solving for $v^*_c$, we get $v^*_c = v^* \cdot \frac{F_d}{F_c} = \frac{v^*}{m}$

Plugging in our values of $T_c=100ms$ and $T_d=500ms$, we can calculate that $F_c=10$, $F_d=2$, and $m=5$.


As an example:$\\$
During data collection, we sampled every $.5$ seconds. If our v* was collected to be $100$ ticks/sample, this means that we expect our car to move $100$ ticks every $.5s$ (assuming constant velocity). Now, our controller's sampling rate is instead $.1$ seconds. If we use the same v* of $100$ ticks/sample, we are now expecting our car to move $100$ ticks in $.1s$, effectively quintupling our velocity! Our controller does not have information about the original sampling period; to account for this, we divide $v^*$ by $5$ when using a $v^*$ value for turning.

**<span style='color:red'> In your delta_reference function, use $\frac{v^*}{m}$ instead of $v^*$. Do not replace the original $v^*$ parameter defined in `CODE BLOCK CON1`!</span>**

### Implementation
- Open `turning.ino`.
- Copy the required code from `closed_loop.ino` into **`CODE BLOCK CON1/2/3`**
    - <span style='color:red'>**Note that your `driveStraight` functions only have one parameter in this program unlike closed_loop.ino, so only copy over the body of the functions!**</span>
- **Complete the function `delta_reference` in `CODE BLOCK CON5`**.
    - Look at `CODE BLOCK CON0` to see how this is used
    - **Use $\frac{v^*}{m}$ instead of $v^*$ in your delta_reference function!**
    - Use the variables `TURN_RADIUS`, `v_star`, `CAR_WIDTH`, and `i` (the time-step).
    - `delta_ss` is used in `CODE BLOCK CON0`, so don't include it here.
- The commands your car follows are defined in **`PREPROGRAMMED PATH`**
    - `NUM_COMMANDS`: number of commands your car will follow. The default is set as 4.
    - `run_times` and `drive_modes`: arrays that store the time and mode of each command
    - The commands your car follows as well as the run time associated with each command is in the 
        - The number of elements in both `drive_modes` and `run_times` is equal to `NUM_COMMANDS`
        - `drive_modes` can take on values: `[DRIVE_STRAIGHT, DRIVE_LEFT, DRIVE_RIGHT]`.
        - Try to tune your `run_time` and `turn_radius` for the turns such that the car turns approximately 90 degrees.
            - **The values in `run_times` are in ms**, so you should have run times that are at least 1000.
            - If your car doesn't turn with the same speed when turning left vs right, you can adjust the run times of the left and right turns to be different values.
        
**Remember**: If you have previously changed out any pins, you will need to update those pin assignments in `turning.ino` accordingly!

When you're done, try it out! S1XT33N should follow the preprogrammed path we provided as a default. If you'd like, feel free to play around with implementing different paths by changing the commands under `PREPROGRAMMED PATH` 

<a id='part3'></a>
## <span style="color:navy">Part 3: Correcting Mechanical Errors</span>

At this stage, your car may already go perfectly straight and turn left and right equally. However, some cars may still turn very slightly even when both encoder measurements are exactly the same ($\delta=constant$). This can be caused by mechanical issues such as axle wobble or mismatch in the wheel sizes. In this case, since the car thinks it's driving straight, it's the best we can do using just the sensors in our control system.

One way we can correct it for this behavior is by instructing the car to turn very slightly when trying to drive straight. If you think your car could benefit from this, implement the function `straight_correction` in `CODE BLOCK CON6` using the variable `STRAIGHT_RADIUS`. Update this value from INFINITY to a suitably large number.

<a id='part4'></a>
## <span style="color:navy">Part 4: Micboard Verification</span>
-----

We will be using the micboard next week for the SVD/PCA lab! Verify that your biasing circuits and front-end circuitry still work as expected, and that we can read in the data with our Arduino.

### Materials:
- Oscilloscope
- Arduino

### Tasks:
1. Probe the output of the micboard with the oscilloscope and ensure you see a signal centered near $1.65\mathrm{V}$. 
2. If necessary, re-tune your mic board so that you see a response with $\approx2.5\mathrm{V}_{pp}$ when you speak close to the microphone.
    - Turn the potentiometer on your mic board **clockwise** to lower gain, **counterclockwise** to increase gain.
3. Probe the output of the non-inverting amplifier that follows your low pass filter. Verify that the output of the amplifier in this circuit is working as expected.
4. Connect pin `A2` to the output of the non-inverting amplifier that follows your low pass filter. Upload `adc_read.ino` to the Arduino.
    - If pin `A2` no longer works, choose another analog read pin (`A0-A5`) and update the code accordingly.
5. In the command line on your computer, navigate into the `adc_read` folder and run `python adc_read.py -D COM###` (substitute ### with the port number of your Arduino from Device Manager).
    - Note: If you are running this on a Mac, open the adc_read.py file and change line 131, `ani = anim.FuncAnimation(fig, update, fargs=(l1,ser, line, data, num_read), interval=0.001*num_read)`, to `ani = anim.FuncAnimation(fig, update, fargs=(l1,ser, line, data, num_read), interval=num_read)`.
6. If you see a graph on your screen with a signal that changes when you make noises at your microphone and is centered in the middle of the graph, that means your Arduino is able to read the output of your mic board circuitry properly and output it to your computer.


# <span style="color:green">CHECKOFF</span>

- Have all questions, code, and plots completed in this notebook.
- Demonstrate your car's preprogrammed path, which **includes all 4 commands (drive straight far, turn left, drive straight close, and turn right).**
- Demonstrate that your **Arduino is reading the mic board circuitry output correctly.**
- **Be prepared to answer conceptual questions about the lab.**

## SAVE ALL YOUR DATA!!

- **Data stored on the lab computers often gets deleted automatically.** Please store it on your personal flash drive or cloud storage like Google Drive, and not on the lab computers! If you used DataHub, the notebook should save through your CalNet ID. **<span style='color:red'> Make sure to back up your .ino files. You will need them for Integration/Final Demo!</span>**